In [1]:
from PIL import Image
import requests
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
import torch
import os
from os import listdir
from os.path import isfile, join
import json



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base", cache_dir="/home/congnguyen/drive/.cache/")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base", cache_dir="/home/congnguyen/drive/.cache/")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base", cache_dir="/home/congnguyen/drive/.cache/", torch_dtype=torch.float64)
model.to(device)

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0): BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BlipEncoderLayer(
          (self_attn): BlipAttention(
     

In [57]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import Dataset
import torch 

class VQADataset(torch.utils.data.Dataset):
    def __init__(self, images, questions, answers, processor):
        self.images = images
        self.questions = questions
        self.answers = answers
        self.processor = processor
        self.max_length = 8
        # self.image_height = 128
        # self.image_width = 128

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        # get image + text
        answers = self.answers[idx]
        questions = self.questions[idx]
        image = Image.open(self.images[idx]).convert("RGB")
        text = self.questions[idx]

        image_encoding = self.processor(image,
                                  do_resize=True,
                                  # size=(self.image_height,self.image_width),
                                  return_tensors="pt")

        encoding = self.processor(
                                  None,
                                  text,
                                  padding="max_length",
                                  truncation=True,
                                  max_length = self.max_length,
                                  return_tensors="pt"
                                  )
        # # remove batch dimension
        for k,v in encoding.items():
            encoding[k] = v.squeeze()
        encoding["pixel_values"] = image_encoding["pixel_values"][0]
        # # add labels
        labels = self.processor.tokenizer.encode(
            answers,
            max_length= self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors='pt'
        )[0]
        encoding["labels"] = labels

        return encoding

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    pixel_values = [item['pixel_values'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]
    # create new batch
    batch = {}
    batch['input_ids'] = torch.stack(input_ids)
    batch['attention_mask'] = torch.stack(attention_mask)
    batch['pixel_values'] = torch.stack(pixel_values)
    batch['labels'] = torch.stack(labels)

    return batch

In [58]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data/train.jsonl", split="train")

questions = [item for item in dataset["question"]]
images = [
        f"data/train_fill_in_blank/train_fill_in_blank/{pid}/image.png" for pid in dataset["pid"]
    ] 
answers = [item for item in dataset["answer"]]

dataset = VQADataset(questions = questions,
                          answers = answers,
                          images = images,
                          processor=processor)
# train_set, val_set = torch.utils.data.random_split(dataset, [13549, 1000])

# test_dataset = VQADataset(questions = questions,
#                           answers = answers,
#                           image_paths = images,
#                           processor=processor)

batch_size = 16
# train_dataloader = DataLoader(train_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)
train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(val_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(k, v.shape)

input_ids torch.Size([16, 8])
attention_mask torch.Size([16, 8])
pixel_values torch.Size([16, 3, 384, 384])
labels torch.Size([16, 8])


In [6]:
from tqdm.notebook import tqdm
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.train()

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0): BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BlipEncoderLayer(
          (self_attn): BlipAttention(
     

In [ ]:
for epoch in range(100):
    print(f"Epoch: {epoch}")
    total_loss = []
    for batch in tqdm(train_dataloader):
        # get the inputs;
        batch = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        with torch.autocast(device_type='cuda', dtype=torch.float64):
            outputs = model(**batch)
        loss = outputs.loss
        total_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    print("Loss:", sum(total_loss))
    if epoch % 5 == 0 or epoch > 90:
        torch.save(model, "./model/BLIP2/checkpoint_"+str(epoch))

Epoch: 0


  0%|          | 0/910 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("./model/BLIPGray/checkpoint_5")
model.to(device)
model.eval()
# prepare image + questio


BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0): BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BlipEncoderLayer(
          (self_attn): BlipAttention(
     

In [14]:
eval_dir = os.path.abspath("data/test_data/test_data/")
eval_data = {}
for it in listdir(eval_dir):
    eval_data.update({it: []})
    for f in listdir(join(eval_dir, it)):
        file = join(eval_dir, it, f)
        eval_data[it].append(file)

def get_data(idx):
    img_url = eval_data[idx][0]
    image = Image.open(img_url).convert("L")
    text_url = eval_data[idx][1]
    textf = open(text_url)
    # returns JSON object as 
    # a dictionary
    data = json.load(textf)
    text = data["question"]
    return image, text

In [15]:
result = {}

for id in eval_data:
    image, text = get_data(id)
    encoding = processor(image, text, return_tensors="pt")
    encoding.to(device)
    # forward pass
    # outputs = model(**encoding)
    out = model.generate(**encoding)
    # logits = outputs.logits
    # idx = logits.argmax(-1).item()
    out = processor.decode(out[0], skip_special_tokens=True).replace(" ","")
    print(out)
    result.update({id: out})

6
4
20
3
6
6
8
3
4
3
10
6
5
6
17
o'clock
2
86
18
2
3
4
3
3
3
6
2
5
2
8
6
5
3
3
6
100
7,355
2
3
78
2


KeyboardInterrupt: 

In [5]:
# result

In [10]:
def write2csv(result, output='./results/blip5ep.csv'):
    import csv
    with open(output, 'w', newline='') as csvfile:
        fieldnames = ['ID', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for idx in result:
            writer.writerow({'ID': idx, 'Label': result[idx]})

In [7]:
write2csv(result)

In [11]:
import csv
def read_csv(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    results = {}
    for row in csvreader:
        results.update({row[0]: row[1]}) 
    return results

In [12]:
blip_result = read_csv("./results/blip5ep.csv")
bl20_result = read_csv("./results/bl20ep.csv")

In [13]:
blip_result

{'58134': '5',
 '25400': '4',
 '93479': '20',
 '47455': '3',
 '63429': '6',
 '81647': '6',
 '79225': '8',
 '49548': '3',
 '88662': '4',
 '73136': '3',
 '106300': '10',
 '79012': '6',
 '61406': '5',
 '71874': '5',
 '15834': '17',
 '6972': "o'clock",
 '18226': '2',
 '48530': '86',
 '4540': '18',
 '36056': '2',
 '55907': '3',
 '56434': '4',
 '97012': '3',
 '83371': '3',
 '88330': '3',
 '89696': '5',
 '30670': '2',
 '34071': '4',
 '3739': '2',
 '64102': '8',
 '45763': '5',
 '45647': '5',
 '59362': '3',
 '69351': '3',
 '79613': '6',
 '47214': '100',
 '96442': '7,337',
 '28871': '2',
 '91102': '3',
 '15927': '68',
 '78430': '2',
 '34745': '53',
 '65273': '5',
 '77680': '2',
 '59461': '20',
 '9848': '3',
 '86653': '3',
 '66515': '11',
 '30762': '16',
 '102143': '5',
 '66690': '2',
 '77822': '5',
 '79519': '1',
 '61748': '2',
 '49142': '2',
 '81022': '1',
 '22668': '3',
 '38547': '6',
 '2116': '3',
 '55183': '8',
 '54099': '5',
 '35951': '70',
 '68435': '11',
 '23974': '11',
 '63748': '5',
 '7

In [14]:
count = 0
all = 0 
compares = {}
false = {}
for key in blip_result:
    all += 1
    if blip_result[key] == bl20_result[key]:
        compares.update({key: blip_result[key]})
        continue
    else:
        false.update({key: [blip_result[key], eval_data[key]]})
        count += 1

In [32]:
write2csv(compares, "./results/false_id.csv")

In [15]:
false

{'96442': ['7,337',
  ['/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/96442/image.png',
   '/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/96442/data.json']],
 '15927': ['68',
  ['/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/15927/image.png',
   '/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/15927/data.json']],
 '19789': ['5,444',
  ['/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/19789/image.png',
   '/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/19789/data.json']],
 '77183': ['3:00',
  ['/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/77183/image.png',
   '/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/77183/data.json']],
 '98725': ['83',
  ['/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/98725/image.png',
   '/home/congnguyen/drive/AdvancedML/VQA/data/test_data/test_data/98725/data.json']],
 '60840': ['54',
  ['/home/congnguyen/drive/Adv

In [19]:
import xlsxwriter


# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook("./results/false_id.xlsx")
worksheet = workbook.add_worksheet()
# Widen the first column to make the text clearer.
worksheet.set_column("D:D", 40)
worksheet.set_column("E:E", 40)

count = 0
for key in false:
    # Insert an image.
    text_url = eval_data[key][1]
    textf = open(text_url)
    # returns JSON object as 
    # a dictionary
    worksheet.set_row(count, 80)
    data = json.load(textf)
    text = data["question"]
    worksheet.write("A"+str(count), key)
    worksheet.write("B"+str(count), blip_result[key])
    worksheet.write("C"+str(count), bl20_result[key])
    worksheet.write("D"+str(count), text)
    worksheet.insert_image("E"+str(count), eval_data[key][0], {"x_scale": 0.5, "y_scale": 0.3})
    count += 1
    

workbook.close()